# Martech - Query Data

## Initialise Connection

In [1]:
from datetime import datetime
from astra_db import AstraSession
from state import OffsetState as offsetstate

astra = AstraSession()

In [2]:
# Prepare Statements

# comm_by_activity_day
get_acty_cap_stmt = astra.session.prepare("""SELECT comm_time, comm_offset_time, comm_offset_count 
                                          FROM actycap_day WHERE 
                                          activity = ? AND comm_date = ? AND comm_time > ?""")

update_acty_cap_stmt = astra.session.prepare("""UPDATE actycap_day
                                             SET comm_offset_time = ?, comm_offset_count = ? 
                                             WHERE activity = ? AND comm_date = ?""")
# comm_by_activity_week
get_acty_cap_week_stmt = astra.session.prepare("""SELECT comm_count FROM actycap_week WHERE 
                                          activity = ? AND comm_date >= ? AND comm_date <= ?""")

## Define Entities

In [3]:
# Define Entities

# customer_id
customer_id_array = [
    '0b6ec30f-8b1a-406f-82f6-89222a017449',
    '8435a9b9-9e88-495c-a297-dfd25d44428b',
    '8e12d570-e47d-43b9-9435-b2020899860b',
    '9e53cd14-0316-46ff-b913-30228fcc5b73',
    'a6e198e0-674d-4a27-ba01-595a78fda2ef',
    'af9e65bd-4474-40c9-8d16-7b82035d3e0f',
    'b191cef3-4624-4e88-9a93-73de506e4be7',
    'bcabe5dd-8675-4a88-94ed-a9faa9473355',
    'cf3a5d68-e66f-4ee7-a2d1-2666c5366540',
    'f68e8bc9-86b7-4c8e-ba74-87d31d27a531',
    'bcabe5dd-8675-4a88-94ed-a9faa9473355',
]

# channel
channel_array = ['email', 'sms', 'push', 'in-app', 'direct']

# category_group
category_group_array = ['ctgy_grp_1', 'ctgy_grp_2', 'ctgy_grp_3', 'ctgy_grp_4', 'ctgy_grp_5']

# category
category_array = ['ctgy_1', 'ctgy_2', 'ctgy_3', 'ctgy_4', 'ctgy_5']

# activity
activity_array = ['credit_card', 'mortgage', 'business', 'retail', 'investment']

## Query Data

In [4]:
cap_name = 'actycap'
activity = 'business'

### Daily

#### Query

In [6]:
# Communications by Activity Day
# SELECT comm_time, comm_offset_time, comm_offset_count FROM comm_by_activity_day 
# WHERE activity = ? AND comm_date = ? AND comm_time > ?

#
# To get the total count, we add comm_offset_count to the number of rows returned


# Set the current day and time buckets
date = datetime.now()
current_date = int(date.strftime('%Y%m%d')) # bucket by day
current_time = int(date.strftime('%H%M%S%f')[:8]) # format time as HHMMSSssss

# Read stored state values for activity from JSON file named state.json
(comm_offset_time, comm_offset_count) = offsetstate.get_state(cap_name, activity)

rows = astra.session.execute(get_acty_cap_stmt, [activity, current_date, comm_offset_time ])
# test if query returned any rows
if not rows:
    print("No rows returned")
    row_count = 0
    total_comm_count = comm_offset_count
else:
    comm_offset_time = rows[0].comm_offset_time or 0
    comm_offset_count = rows[0].comm_offset_count or 0
    comm_time = rows[0].comm_time or 0
    row_count = sum(1 for _ in rows)  # Count rows
    total_comm_count = comm_offset_count + row_count

print(f"Total Count: {total_comm_count},  Row Count: {row_count}")
#print(f"Comm Time Bucket: {current_time}  Offset Time: { comm_offset_time}  Offset Count: {comm_offset_count}")

Total Count: 3,  Row Count: 3


#### Update

In [ ]:
# Update - Communications by Activity Day
# SET comm_offset_time = ?, comm_offset_count = ? WHERE activity = ? AND comm_date = ?
#
# comm_offset_time will be the last comm_time value read
# comm_offset_count will be the number of comm_time values counted up to that time.

rows = astra.session.execute(update_acty_cap_stmt, [comm_time, total_comm_count, activity, current_date])

# Store State for next run
offsetstate.set_state(cap_name, activity, comm_time, total_comm_count)

### Weekly

In [ ]:
activity = 'business'
start_date = 20250303
end_date = 20250310

rows = astra.session.execute(get_acty_cap_week_stmt, [activity, start_date, end_date])

if not rows:
    print("No rows returned")
    count = 0
else:
    count = sum(row.comm_count for row in rows)

print(f"Count: {count}")

## Generate Daily Summaries

In [ ]:
current_date = 20250305

activity_array = ['business']
for activity in activity_array:
    rows = astra.session.execute(f"SELECT comm_time, comm_offset_time, comm_offset_count FROM actycap_day WHERE \
                                activity = '{activity}' AND comm_date = {current_date}")
    if not rows:
        print("No rows returned")
        row_count = 0
        continue
    else:
        row_count = sum(1 for _ in rows)  # Count rows

    print(f"Activity: {activity} Row Count: {row_count}")

    # Insert summary data for the day
    #    activity text,
    #    comm_date int,
    #    comm_count int,

    astra.session.execute(f"INSERT INTO actycap_week (activity, comm_date, comm_count) \
                        VALUES ('{activity}', {current_date}, {row_count})")

## Cleanup

In [7]:
astra.shutdown()